# Multiclass Classification for Transactions

For this notebook we will be looking to classify a transaction dataset of transactions into a number of categories that we have predefined.

The approach we'll be taking in this notebook:
- **Zero-shot Classification:** First we'll do zero shot classification to put transactions in one of five named buckets using only a prompt for guidance

## Setup

In [1]:
import openai
from openai import OpenAI
import pandas as pd
import json
import os

In [14]:
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key
client = OpenAI(api_key=api_key)
COMPLETIONS_MODEL = "gpt-4o"

In [75]:
transactions = pd.read_csv("../data/raw/transactie-historie_final.csv", sep=";")
len(transactions)

65

In [77]:
column_names = [
    "date",
    "account",
    "supplier account",
    "supplier name",
    "Unnamed: 4",
    "Unnamed: 5",
    "Unnamed: 6",
    "currency",
    "code",
    "currency 1",
    "transaction amount",
    "date1",
    "date2",
    "code1",
    "code2",
    "code3",
    "Unnamed: 16",
    "description",
    "number",
]

# Assign the new column names to the DataFrame
transactions.columns = column_names
transactions = transactions.drop(
    columns=[
        "Unnamed: 4",
        "Unnamed: 5",
        "Unnamed: 6",
        "code",
        "currency 1",
        "date1",
        "date2",
        "code1",
        "code2",
        "code3",
        "Unnamed: 16",
        "number",
    ]
)

In [ ]:
transactions["full_description"] = (
    transactions["supplier name"].fillna("Unknown Supplier").astype(str)
    + " "
    + transactions["description"].fillna("Description")
)

In [83]:
def request_completion(prompt):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
    ]

    completion_response = openai.chat.completions.create(
        messages=messages,
        model=COMPLETIONS_MODEL,
        temperature=0,
        max_tokens=5,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )

    return completion_response.choices[0].message.content


def classify_transaction(transaction, prompt):
    prompt = prompt.replace("DESCRIPTION_TEXT", transaction["full_description"])
    prompt = prompt.replace("TRANSACTION_VALUE", str(transaction["transaction amount"]))

    classification = request_completion(prompt)

    return classification


# This function takes your training and validation outputs from the prepare_data function of the Finetuning API, and
# confirms that each have the same number of classes.
# If they do not have the same number of classes the fine-tune will fail and return an error


def check_finetune_classes(train_file, valid_file):
    train_classes = set()
    valid_classes = set()
    with open(train_file, "r") as json_file:
        json_list = list(json_file)
        print(len(json_list))

    for json_str in json_list:
        result = json.loads(json_str)
        train_classes.add(result["completion"])
        # print(f"result: {result['completion']}")
        # print(isinstance(result, dict))

    with open(valid_file, "r") as json_file:
        json_list = list(json_file)
        print(len(json_list))

    for json_str in json_list:
        result = json.loads(json_str)
        valid_classes.add(result["completion"])
        # print(f"result: {result['completion']}")
        # print(isinstance(result, dict))

    if len(train_classes) == len(valid_classes):
        print("All good")

    else:
        print("Classes do not match, please prepare data again")


## Zero-shot Classification

We'll first assess the performance of the base models at classifying these transactions using a simple prompt. We'll provide the model with 5 categories and a catch-all of "Could not classify" for ones that it cannot place.

In [84]:
zero_shot_prompt = """You are a data expert on personal expenses
You are analysing all transactions for a monthly expense report. Each transaction must be classified into one of the 5 following categories: Utility bills, Health and Beauty, Shopping, Food, Housing.
Your answer must be short and concise and CONTAIN ONLY the CATEGORY, NOTHING ELSE.

Transaction information:

Supplier: SUPPLIER_NAME
Description: DESCRIPTION_TEXT
Value: TRANSACTION_VALUE



The category is: """


In [85]:
# Get a test transaction
transaction = transactions.iloc[0]

# Interpolate the values into the prompt
# prompt = zero_shot_prompt.replace("SUPPLIER_NAME", transaction["supplier name"])
prompt = zero_shot_prompt.replace("DESCRIPTION_TEXT", transaction["full_description"])
prompt = prompt.replace("TRANSACTION_VALUE", str(transaction["transaction amount"]))

# Use our completion function to return a prediction
completion_response = request_completion(prompt)
completion_response


'Housing'

Our first attempt is correct, M & J Ballantyne Ltd are a house builder and the work they performed is indeed Building Improvement.

Lets expand the sample size to 25 and see how it performs, again with just a simple prompt to guide it

In [86]:
transactions["Classification"] = transactions.apply(
    lambda x: classify_transaction(x, zero_shot_prompt), axis=1
)


In [87]:
transactions["Classification"].value_counts()


Classification
Food                 34
Shopping             12
Health and Beauty     7
Housing               6
Utility bills         6
Name: count, dtype: int64

In [ ]:
df = transactions
df["combined"] = (
    "; Description: "
    + df["full_description"].str.strip()
    + "; Value: "
    + str(df["transaction amount"]).strip()
)

In [95]:
def get_embedding(text: str, model="text-embedding-3-small") -> list[float]:
    return client.embeddings.create(input=[text], model=model).data[0].embedding


df["embedding"] = df.combined.apply(lambda x: get_embedding(x))


In [97]:
from datetime import datetime

now = datetime.now()
df.to_csv(f"../data/raw/transaction_labelled_{now}.csv")
